In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...


LoadError: LoadError: PyError (:PyImport_ImportModule) <class 'gurobipy.GurobiError'>
GurobiError()
  File "GLS_Apr_weekday_PM.py", line 139, in <module>
    xi_list = GLS(x, A, L)
  File "GLS_Apr_weekday_PM.py", line 41, in GLS
    model = Model("OD_matrix_estimation")
  File "model.pxi", line 53, in gurobipy.Model.__init__ (../../src/python/gurobipy.c:38683)
  File "gurobi.pxi", line 15, in gurobipy.gurobi._getdefaultenv (../../src/python/gurobipy.c:82297)
  File "env.pxi", line 28, in gurobipy.Env.__init__ (../../src/python/gurobipy.c:5561)

while loading In[1], in expression starting on line 427

In [2]:
xi_list = GLS_Apr_weekday_PM.xi_list
P = GLS_Apr_weekday_PM.P
L = GLS_Apr_weekday_PM.L  # dimension of xi

LoadError: LoadError: UndefVarError: GLS_Apr_weekday_PM not defined
while loading In[2], in expression starting on line 1

In [3]:
size(P, 1), size(P, 2)

LoadError: LoadError: UndefVarError: P not defined
while loading In[3], in expression starting on line 1

In [4]:
P

LoadError: LoadError: UndefVarError: P not defined
while loading In[4], in expression starting on line 1

In [5]:
using JuMP

mGLSJulia = Model()

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)

LoadError: LoadError: UndefVarError: P not defined
while loading In[5], in expression starting on line 5

In [6]:
getvalue(lam)

LoadError: LoadError: UndefVarError: lam not defined
while loading In[6], in expression starting on line 1

In [7]:
getobjectivevalue(mGLSJulia)

0

In [8]:
GLS_Apr_weekday_PM.saveDemandVec(getvalue(lam))

LoadError: LoadError: UndefVarError: GLS_Apr_weekday_PM not defined
while loading In[8], in expression starting on line 1

In [9]:
getvalue(p)

LoadError: LoadError: UndefVarError: p not defined
while loading In[9], in expression starting on line 1

In [10]:
route_choice_matrix_Apr_PM = getvalue(p)

LoadError: LoadError: UndefVarError: p not defined
while loading In[10], in expression starting on line 1

In [11]:
sum(route_choice_matrix_Apr_PM[1, :])

LoadError: LoadError: UndefVarError: route_choice_matrix_Apr_PM not defined
while loading In[11], in expression starting on line 1

In [12]:
route_choice_matrix_dict_Apr_PM = Dict()

for row_idx = 1:size(route_choice_matrix_Apr_PM)[1]
    for col_idx = 1:size(route_choice_matrix_Apr_PM)[2]
        route_choice_matrix_dict_Apr_PM[(row_idx, col_idx)] = route_choice_matrix_Apr_PM[row_idx, col_idx]
    end
end

LoadError: LoadError: UndefVarError: route_choice_matrix_Apr_PM not defined
while loading In[12], in expression starting on line 3

In [13]:
outfile = open("../temp_files/route_choice_CDC_Apr_PM.json", "w")

JSON.print(outfile, route_choice_matrix_dict_Apr_PM)

close(outfile)

# Route flow analysis (see [Eq. (9) in M. Hazelton (2000); Page 556])

$\textbf{P}'\textbf{g}$ equals the route flow vector. It seems hard to uniquely determine a route flow vector corresponding to a social link flow vector.